In [1]:
!pip install mysql-connector-python pandas



In [73]:
import numpy as np

In [108]:
import mysql.connector
import pandas as pd

# Database connection details
config = {
    'user': 'root',
    'password': 'Aa12369857826',
    'host': 'localhost',
    'database': 'Taxi'
}

# Connect to the database
try:
    conn = mysql.connector.connect(**config)
    cursor = conn.cursor()
    print("Connection successful")
except mysql.connector.Error as err:
    print(f"Error: {err}")


Connection successful


In [28]:
# Specify data types for each column
dtype_spec = {
    'VendorID': 'int64',
    'tpep_pickup_datetime': 'str',
    'tpep_dropoff_datetime': 'str',
    'passenger_count': 'float64',
    'trip_distance': 'float64',
    'RatecodeID': 'float64',
    'store_and_fwd_flag': 'str',
    'PULocationID': 'int64',
    'DOLocationID': 'int64',
    'payment_type': 'int64',
    'fare_amount': 'float64',
    'extra': 'float64',
    'mta_tax': 'float64',
    'tip_amount': 'float64',
    'tolls_amount': 'float64',
    'improvement_surcharge': 'float64',
    'total_amount': 'float64',
    'congestion_surcharge': 'float64',
    'Airport_fee': 'float64'
}

# Load data from CSV files
yellow_tripdata = pd.read_csv('yellow_tripdata_Mar.csv', dtype=dtype_spec, low_memory=False)
# taxi_zone_lookup = pd.read_csv('/home/coder/project/Advance_DB_Mid/taxi_zone_lookup.csv')


# Fill missing numeric values with 0 (or any appropriate default)
numeric_columns = ['passenger_count', 'trip_distance', 'RatecodeID', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount', 'congestion_surcharge', 'Airport_fee']
yellow_tripdata[numeric_columns] = yellow_tripdata[numeric_columns].fillna(0)

# Fill missing categorical values with a default string or handle as needed
categorical_columns = ['VendorID', 'store_and_fwd_flag', 'PULocationID', 'DOLocationID', 'payment_type']
yellow_tripdata[categorical_columns] = yellow_tripdata[categorical_columns].fillna('N')

# Save the cleaned data back to CSV
yellow_tripdata.to_csv('yellow_tripdata_Mar.csv', index=False)

print(yellow_tripdata.head())
# print(taxi_zone_lookup.head())
max_values = yellow_tripdata[['passenger_count', 'trip_distance', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount', 'congestion_surcharge', 'Airport_fee']].max()

print(max_values)

   VendorID tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  \
0         1  2024-03-01 00:18:51   2024-03-01 00:23:45              0.0   
1         1  2024-03-01 00:26:00   2024-03-01 00:29:06              0.0   
2         2  2024-03-01 00:09:22   2024-03-01 00:15:24              1.0   
3         2  2024-03-01 00:33:45   2024-03-01 00:39:34              1.0   
4         1  2024-03-01 00:05:43   2024-03-01 00:26:22              0.0   

   trip_distance  RatecodeID store_and_fwd_flag  PULocationID  DOLocationID  \
0           1.30         1.0                  N           142           239   
1           1.10         1.0                  N           238            24   
2           0.86         1.0                  N           263            75   
3           0.82         1.0                  N           164           162   
4           4.90         1.0                  N           263             7   

   payment_type  fare_amount  extra  mta_tax  tip_amount  tolls_amount  \


In [35]:

csv_file_path = 'yellow_tripdata_Mar.csv'
yellow_tripdata_miss= pd.read_csv(csv_file_path, low_memory=False)

# Ensure the datetime columns are treated as strings to handle empty string checks
yellow_tripdata_miss['tpep_pickup_datetime'] = yellow_tripdata_miss['tpep_pickup_datetime'].astype(str)
yellow_tripdata_miss['tpep_dropoff_datetime'] = yellow_tripdata_miss['tpep_dropoff_datetime'].astype(str)

# Identify rows with missing pickup_datetime or dropoff_datetime
missing_datetime_rows = yellow_tripdata_miss[
    (yellow_tripdata_miss['tpep_pickup_datetime'].isnull()) | 
    (yellow_tripdata_miss['tpep_pickup_datetime'].str.strip() == '') |
    (yellow_tripdata_miss['tpep_dropoff_datetime'].isnull()) | 
    (yellow_tripdata_miss['tpep_dropoff_datetime'].str.strip() == '')
]

# Print the rows with missing datetime values
print("Rows with missing pickup_datetime or dropoff_datetime:")
print(missing_datetime_rows)

create_boroughs_table = """
CREATE TABLE IF NOT EXISTS Boroughs (
    borough_id INT PRIMARY KEY AUTO_INCREMENT,
    borough_name VARCHAR(50) UNIQUE NOT NULL
);
"""

create_servicezones_table = """
CREATE TABLE IF NOT EXISTS ServiceZones (
    service_zone_id INT PRIMARY KEY AUTO_INCREMENT,
    service_zone_name VARCHAR(50) UNIQUE NOT NULL
);
"""

create_zones_table = """
CREATE TABLE IF NOT EXISTS Zones (
    zone_id INT PRIMARY KEY AUTO_INCREMENT,
    zone_name VARCHAR(100) UNIQUE NOT NULL,
    borough_id INT NOT NULL,
    service_zone_id INT NOT NULL,
    FOREIGN KEY (borough_id) REFERENCES Boroughs(borough_id),
    FOREIGN KEY (service_zone_id) REFERENCES ServiceZones(service_zone_id)
);
"""

create_locations_table = """
CREATE TABLE IF NOT EXISTS Locations (
    location_id INT PRIMARY KEY,
    zone_id INT NOT NULL,
    FOREIGN KEY (zone_id) REFERENCES Zones(zone_id)
);
"""

create_vendors_table = """
CREATE TABLE IF NOT EXISTS Vendors (
    vendor_id INT PRIMARY KEY,
    vendor_name VARCHAR(50) NOT NULL
);
"""

create_paymenttypes_table = """
CREATE TABLE IF NOT EXISTS PaymentTypes (
    payment_type_id INT PRIMARY KEY,
    payment_type_description VARCHAR(50) NOT NULL
);
"""

create_ratecodes_table = """
CREATE TABLE IF NOT EXISTS RateCodes (
    ratecode_id INT PRIMARY KEY,
    ratecode_description VARCHAR(50) NOT NULL
);
"""

create_trips_table = """
CREATE TABLE IF NOT EXISTS Trips (
    trip_id INT PRIMARY KEY AUTO_INCREMENT,
    vendor_id INT NULL,
    pickup_datetime DATETIME NOT NULL,
    dropoff_datetime DATETIME NOT NULL,
    passenger_count INT NULL,
    trip_distance DECIMAL(9,2) NULL,  -- Maximum value 176836.30
    ratecode_id INT NULL,
    store_and_fwd_flag CHAR(1) NULL,
    pickup_location_id INT NULL,
    dropoff_location_id INT NULL,
    payment_type INT NULL,
    fare_amount DECIMAL(6,2) NULL,  -- Maximum value 900.00
    extra DECIMAL(4,2) NULL,  -- Maximum value 14.25
    mta_tax DECIMAL(4,2) NULL,  -- Maximum value 35.84
    tip_amount DECIMAL(6,2) NULL,  -- Maximum value 999.99
    tolls_amount DECIMAL(6,2) NULL,  -- Maximum value 163.00
    improvement_surcharge DECIMAL(3,2) NULL,  -- Maximum value 1.00
    total_amount DECIMAL(7,2) NULL,  -- Maximum value 1021.99
    congestion_surcharge DECIMAL(3,2) NULL,  -- Maximum value 2.50
    airport_fee DECIMAL(3,2) NULL,  -- Maximum value 1.75,
    FOREIGN KEY (vendor_id) REFERENCES Vendors(vendor_id),
    FOREIGN KEY (pickup_location_id) REFERENCES Locations(location_id),
    FOREIGN KEY (dropoff_location_id) REFERENCES Locations(location_id),
    FOREIGN KEY (payment_type) REFERENCES PaymentTypes(payment_type_id),
    FOREIGN KEY (ratecode_id) REFERENCES RateCodes(ratecode_id)
);
"""

# Execute the create table commands
tables = [
    create_boroughs_table,
    create_servicezones_table,
    create_zones_table,
    create_locations_table,
    create_vendors_table,
    create_paymenttypes_table,
    create_ratecodes_table,
    create_trips_table
]

for table in tables:
    cursor.execute(table)
    conn.commit()

print("Tables created successfully")


Rows with missing pickup_datetime or dropoff_datetime:
Empty DataFrame
Columns: [VendorID, tpep_pickup_datetime, tpep_dropoff_datetime, passenger_count, trip_distance, RatecodeID, store_and_fwd_flag, PULocationID, DOLocationID, payment_type, fare_amount, extra, mta_tax, tip_amount, tolls_amount, improvement_surcharge, total_amount, congestion_surcharge, Airport_fee]
Index: []


In [5]:
# try:
#     # Load data from CSV files
#     taxi_zone_lookup = pd.read_csv('taxi_zone_lookup.csv')

#     # Inspect data for null values
#     print(taxi_zone_lookup.isnull().sum())

#     # Handle null values by filling them with a default value, e.g., 'Unknown'
#     taxi_zone_lookup['Borough'].fillna('Unknown', inplace=True)
#     taxi_zone_lookup['Zone'].fillna('Unknown', inplace=True)
#     taxi_zone_lookup['service_zone'].fillna('Unknown', inplace=True)

#     # Insert data into Boroughs and ServiceZones tables
#     boroughs = taxi_zone_lookup['Borough'].drop_duplicates()
#     service_zones = taxi_zone_lookup['service_zone'].drop_duplicates()

#     for borough in boroughs:
#         cursor.execute("INSERT INTO Boroughs (borough_name) VALUES (%s) ON DUPLICATE KEY UPDATE borough_name=borough_name", (borough,))
    
#     for service_zone in service_zones:
#         cursor.execute("INSERT INTO ServiceZones (service_zone_name) VALUES (%s) ON DUPLICATE KEY UPDATE service_zone_name=service_zone_name", (service_zone,))
    
#     conn.commit()
#     print("Boroughs and ServiceZones tables populated successfully")

#     # Fetch ids for Boroughs and ServiceZones
#     cursor.execute("SELECT borough_id, borough_name FROM Boroughs")
#     borough_dict = {row[1]: row[0] for row in cursor.fetchall()}

#     cursor.execute("SELECT service_zone_id, service_zone_name FROM ServiceZones")
#     service_zone_dict = {row[1]: row[0] for row in cursor.fetchall()}

#     # Insert data into Zones table
#     zones = taxi_zone_lookup[['Zone', 'Borough', 'service_zone']].drop_duplicates()
#     for _, row in zones.iterrows():
#         borough_id = borough_dict[row['Borough']]
#         service_zone_id = service_zone_dict[row['service_zone']]
#         cursor.execute("INSERT INTO Zones (zone_name, borough_id, service_zone_id) VALUES (%s, %s, %s) ON DUPLICATE KEY UPDATE zone_name=zone_name", (row['Zone'], borough_id, service_zone_id))
    
#     conn.commit()
#     print("Zones table populated successfully")

#     # Fetch ids for Zones
#     cursor.execute("SELECT zone_id, zone_name FROM Zones")
#     zone_dict = {row[1]: row[0] for row in cursor.fetchall()}

#     # Insert data into Locations table
#     for _, row in taxi_zone_lookup.iterrows():
#         zone_id = zone_dict[row['Zone']]
#         cursor.execute("INSERT INTO Locations (location_id, zone_id) VALUES (%s, %s)", (row['LocationID'], zone_id))
    
#     conn.commit()
#     print("Locations table populated successfully")
# except mysql.connector.Error as err:
#     print(f"Error: {err}")




try:
    # Load data from CSV files
    taxi_zone_lookup = pd.read_csv('taxi_zone_lookup.csv')

    # Inspect data for null values
    print(taxi_zone_lookup.isnull().sum())

    # Handle null values by filling them with a default value, e.g., 'Unknown'
    taxi_zone_lookup['Borough'].fillna('Unknown', inplace=True)
    taxi_zone_lookup['Zone'].fillna('Unknown', inplace=True)
    taxi_zone_lookup['service_zone'].fillna('Unknown', inplace=True)

    # Normalize Boroughs
    # Insert unique boroughs into the Boroughs table, handling duplicates with ON DUPLICATE KEY UPDATE
    boroughs = taxi_zone_lookup['Borough'].drop_duplicates()
    for borough in boroughs:
        cursor.execute("INSERT INTO Boroughs (borough_name) VALUES (%s) ON DUPLICATE KEY UPDATE borough_name=borough_name", (borough,))
    
    # Normalize Service Zones
    # Insert unique service zones into the ServiceZones table, handling duplicates with ON DUPLICATE KEY UPDATE
    service_zones = taxi_zone_lookup['service_zone'].drop_duplicates()
    for service_zone in service_zones:
        cursor.execute("INSERT INTO ServiceZones (service_zone_name) VALUES (%s) ON DUPLICATE KEY UPDATE service_zone_name=service_zone_name", (service_zone,))
    
    # Commit the transactions to save changes
    conn.commit()
    print("Boroughs and ServiceZones tables populated successfully")

    # Fetch ids for Boroughs to create a mapping dictionary
    cursor.execute("SELECT borough_id, borough_name FROM Boroughs")
    borough_dict = {row[1]: row[0] for row in cursor.fetchall()}

    # Fetch ids for ServiceZones to create a mapping dictionary
    cursor.execute("SELECT service_zone_id, service_zone_name FROM ServiceZones")
    service_zone_dict = {row[1]: row[0] for row in cursor.fetchall()}

    # Normalize Zones
    # Insert unique zones into the Zones table, linking them to borough and service zone ids
    zones = taxi_zone_lookup[['Zone', 'Borough', 'service_zone']].drop_duplicates()
    for _, row in zones.iterrows():
        borough_id = borough_dict[row['Borough']]
        service_zone_id = service_zone_dict[row['service_zone']]
        cursor.execute("INSERT INTO Zones (zone_name, borough_id, service_zone_id) VALUES (%s, %s, %s) ON DUPLICATE KEY UPDATE zone_name=zone_name", (row['Zone'], borough_id, service_zone_id))
    
    # Commit the transactions to save changes
    conn.commit()
    print("Zones table populated successfully")

    # Fetch ids for Zones to create a mapping dictionary
    cursor.execute("SELECT zone_id, zone_name FROM Zones")
    zone_dict = {row[1]: row[0] for row in cursor.fetchall()}

    # Populate Locations table
    # Insert data into Locations table, linking each LocationID to its corresponding zone id
    for _, row in taxi_zone_lookup.iterrows():
        zone_id = zone_dict[row['Zone']]
        cursor.execute("INSERT INTO Locations (location_id, zone_id) VALUES (%s, %s)", (row['LocationID'], zone_id))
    
    # Commit the transactions to save changes
    conn.commit()
    print("Locations table populated successfully")
except mysql.connector.Error as err:
    print(f"Error: {err}")


LocationID      0
Borough         1
Zone            1
service_zone    2
dtype: int64
Boroughs and ServiceZones tables populated successfully
Zones table populated successfully
Locations table populated successfully


In [6]:
def insert_vendors(cursor):
    vendors = {
        1: 'Creative Mobile Technologies, LLC',
        2: 'VeriFone Inc.'
    }
    for vendor_id, vendor_name in vendors.items():
        cursor.execute(
            "INSERT INTO Vendors (vendor_id, vendor_name) VALUES (%s, %s) ON DUPLICATE KEY UPDATE vendor_name=VALUES(vendor_name)",
            (vendor_id, vendor_name)
        )
    conn.commit()

def insert_paymenttypes(cursor):
    payment_types = {
        1: 'Credit Card',
        2: 'Cash',
        3: 'No charge',
        4: 'Dispute',
        5: 'Unknown',
        6: 'Voided trip'
    }
    for payment_type_id, payment_type_description in payment_types.items():
        cursor.execute(
            "INSERT INTO PaymentTypes (payment_type_id, payment_type_description) VALUES (%s, %s) ON DUPLICATE KEY UPDATE payment_type_description=VALUES(payment_type_description)",
            (payment_type_id, payment_type_description)
        )
    conn.commit()

def insert_ratecodes(cursor):
    rate_codes = {
        1: 'Standard rate',
        2: 'JFK',
        3: 'Newark',
        4: 'Nassau or Westchester',
        5: 'Negotiated fare',
        6: 'Group ride'
    }
    for ratecode_id, ratecode_description in rate_codes.items():
        cursor.execute(
            "INSERT INTO RateCodes (ratecode_id, ratecode_description) VALUES (%s, %s) ON DUPLICATE KEY UPDATE ratecode_description=VALUES(ratecode_description)",
            (ratecode_id, ratecode_description)
        )
    conn.commit()

# Insert data into tables
insert_vendors(cursor)
# insert_locations(cursor, taxi_zone_lookup)
insert_paymenttypes(cursor)
insert_ratecodes(cursor)

print("Reference tables populated successfully")



   VendorID tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  \
0         1  2024-03-01 00:18:51   2024-03-01 00:23:45              0.0   
1         1  2024-03-01 00:26:00   2024-03-01 00:29:06              0.0   
2         2  2024-03-01 00:09:22   2024-03-01 00:15:24              1.0   
3         2  2024-03-01 00:33:45   2024-03-01 00:39:34              1.0   
4         1  2024-03-01 00:05:43   2024-03-01 00:26:22              0.0   

   trip_distance  RatecodeID store_and_fwd_flag  PULocationID  DOLocationID  \
0           1.30         1.0                  N           142           239   
1           1.10         1.0                  N           238            24   
2           0.86         1.0                  N           263            75   
3           0.82         1.0                  N           164           162   
4           4.90         1.0                  N           263             7   

   payment_type  fare_amount  extra  mta_tax  tip_amount  tolls_amount  \


In [38]:
try:
    # Disable foreign key checks and non-primary indexes
    cursor.execute("SET foreign_key_checks = 0;")
    cursor.execute("ALTER TABLE Trips DISABLE KEYS;")
    conn.commit()

    # Load data using LOAD DATA INFILE
    load_query = """
    LOAD DATA INFILE 'C:/ProgramData/MySQL/MySQL Server 8.0/Uploads/yellow_tripdata_Mar.csv'
    INTO TABLE Trips
    FIELDS TERMINATED BY ',' 
    ENCLOSED BY '"' 
    LINES TERMINATED BY '\n'
    IGNORE 1 ROWS
    (vendor_id, pickup_datetime, dropoff_datetime, passenger_count, trip_distance, ratecode_id, store_and_fwd_flag, 
    pickup_location_id, dropoff_location_id, payment_type, fare_amount, extra, mta_tax, tip_amount, tolls_amount, 
    improvement_surcharge, total_amount, congestion_surcharge, Airport_fee)
    """
    
    cursor.execute(load_query)
    conn.commit()
    print("Data loaded successfully")

except mysql.connector.Error as err:
    print(f"Error during data loading: {err}")
    conn.rollback()  # Rollback the transaction on error

finally:
    # Re-enable indexes and foreign key checks
    cursor.execute("ALTER TABLE Trips ENABLE KEYS;")
    cursor.execute("SET foreign_key_checks = 1;")
    conn.commit()


Data loaded successfully


In [109]:
# # Load the dataset
# file_path = 'NYC_Mar_Weather.csv'
# df = pd.read_csv(file_path)

# # Create Datetime Table data
# datetimes = df[['datetime']].drop_duplicates().reset_index(drop=True)
# datetimes['datetime_id'] = datetimes.index + 1

# # Create Conditions Table data
# conditions = df[['conditions']].drop_duplicates().reset_index(drop=True)
# conditions['condition_id'] = conditions.index + 1

# # Create Icons Table data
# icons = df[['icon']].drop_duplicates().reset_index(drop=True)
# icons['icon_id'] = icons.index + 1


# # Merge data to create the Weather Table data
# df_merged = df.merge(datetimes, on='datetime').merge(conditions, on='conditions').merge(icons, on='icon').merge(preciptypes, on='preciptype')
# df_merged.drop(columns=['datetime', 'conditions', 'icon', 'preciptype'], inplace=True)

# # Reorder columns for better readability
# weather_columns = ['datetime_id', 'condition_id', 'icon_id'] + [col for col in df_merged.columns if col not in ['datetime_id', 'condition_id', 'icon_id']]
# df_weather = df_merged[weather_columns]
# df_weather = df_weather.replace({np.nan: None})
# df_weather = df_weather.drop(columns=['name'])
# df_weather = df_weather.drop(columns=['preciptype_id'])



# Load the dataset
file_path = 'NYC_Mar_Weather.csv'
df = pd.read_csv(file_path)

# Normalize Datetime
# Create a unique Datetime table with a unique identifier for each datetime
datetimes = df[['datetime']].drop_duplicates().reset_index(drop=True)
datetimes['datetime_id'] = datetimes.index + 1

# Normalize Conditions
# Create a unique Conditions table with a unique identifier for each condition
conditions = df[['conditions']].drop_duplicates().reset_index(drop=True)
conditions['condition_id'] = conditions.index + 1

# Normalize Icons
# Create a unique Icons table with a unique identifier for each icon
icons = df[['icon']].drop_duplicates().reset_index(drop=True)
icons['icon_id'] = icons.index + 1

# Normalize Preciptypes
# Create a unique Preciptypes table with a unique identifier for each preciptype
preciptypes = df[['preciptype']].drop_duplicates().reset_index(drop=True)
preciptypes['preciptype_id'] = preciptypes.index + 1

# Merge data to create the Weather Table data
# Merge the original data with the normalized tables to replace string values with their respective IDs
df_merged = df.merge(datetimes, on='datetime').merge(conditions, on='conditions').merge(icons, on='icon').merge(preciptypes, on='preciptype')

# Drop the original string columns as they have been replaced by their IDs
df_merged.drop(columns=['datetime', 'conditions', 'icon', 'preciptype'], inplace=True)

# Reorder columns for better readability
# Place the ID columns at the beginning for clarity
weather_columns = ['datetime_id', 'condition_id', 'icon_id'] + [col for col in df_merged.columns if col not in ['datetime_id', 'condition_id', 'icon_id']]
df_weather = df_merged[weather_columns]

# Replace NaN values with None for better handling in databases
df_weather = df_weather.replace({np.nan: None})

# Drop columns that are not needed
df_weather = df_weather.drop(columns=['name'])
df_weather = df_weather.drop(columns=['preciptype_id'])



In [111]:
# Print the columns to ensure they match
print("Weather DataFrame columns:")
print(df_weather.columns)

# Print the number of columns and a sample row to debug
print(f"Number of columns in df_weather: {len(df_weather.columns)}")
print(f"Sample row from df_weather: {df_weather.iloc[0].to_dict()}")

Weather DataFrame columns:
Index(['datetime_id', 'condition_id', 'icon_id', 'temp', 'feelslike', 'dew',
       'humidity', 'precip', 'precipprob', 'snow', 'snowdepth', 'windgust',
       'windspeed', 'winddir', 'sealevelpressure', 'cloudcover', 'visibility',
       'solarradiation', 'solarenergy', 'uvindex', 'severerisk', 'stations'],
      dtype='object')
Number of columns in df_weather: 22
Sample row from df_weather: {'datetime_id': 1, 'condition_id': 1, 'icon_id': 1, 'temp': 1.7, 'feelslike': 1.7, 'dew': -11.2, 'humidity': 37.92, 'precip': 0.0, 'precipprob': 0, 'snow': 0, 'snowdepth': 0, 'windgust': 18.4, 'windspeed': 0.6, 'winddir': 359, 'sealevelpressure': 1031.8, 'cloudcover': 0.0, 'visibility': 16.0, 'solarradiation': 0, 'solarenergy': 0.0, 'uvindex': 0, 'severerisk': 10, 'stations': '72505394728,72055399999,KLGA,KJRB,F1417,KNYC,72503014732'}


In [112]:
# Verify that there are no nan values in the final data
print("Checking for NaN values in df_weather...")
print(df_weather.isnull().sum())

Checking for NaN values in df_weather...
datetime_id         0
condition_id        0
icon_id             0
temp                0
feelslike           0
dew                 0
humidity            0
precip              0
precipprob          0
snow                0
snowdepth           0
windgust            0
windspeed           0
winddir             0
sealevelpressure    0
cloudcover          0
visibility          0
solarradiation      0
solarenergy         0
uvindex             0
severerisk          0
stations            0
dtype: int64


In [114]:
# Create Datetime Table
cursor.execute("""
CREATE TABLE IF NOT EXISTS Datetimes (
    datetime_id INT AUTO_INCREMENT PRIMARY KEY,
    datetime DATETIME NOT NULL
);
""")
conn.commit()

# Create Conditions Table
cursor.execute("""
CREATE TABLE IF NOT EXISTS Conditions (
    condition_id INT AUTO_INCREMENT PRIMARY KEY,
    conditions VARCHAR(255) NOT NULL
);
""")
conn.commit()

# Create Icons Table
cursor.execute("""
CREATE TABLE IF NOT EXISTS Icons (
    icon_id INT AUTO_INCREMENT PRIMARY KEY,
    icon VARCHAR(255) NOT NULL
);
""")
conn.commit()


# Create Weather Table
cursor.execute("""
CREATE TABLE IF NOT EXISTS Weather (
    weather_id INT AUTO_INCREMENT PRIMARY KEY,
    datetime_id INT,
    condition_id INT,
    icon_id INT,
    temp FLOAT,
    feelslike FLOAT,
    dew FLOAT,
    humidity FLOAT,
    precip FLOAT,
    precipprob FLOAT,
    snow FLOAT,
    snowdepth FLOAT,
    windgust FLOAT,
    windspeed FLOAT,
    winddir FLOAT,
    sealevelpressure FLOAT,
    cloudcover FLOAT,
    visibility FLOAT,
    solarradiation FLOAT,
    solarenergy FLOAT,
    uvindex INT,
    severerisk INT,
    stations TEXT,
    FOREIGN KEY (datetime_id) REFERENCES Datetimes(datetime_id),
    FOREIGN KEY (condition_id) REFERENCES Conditions(condition_id),
    FOREIGN KEY (icon_id) REFERENCES Icons(icon_id)
);
""")
conn.commit()



In [115]:

# Insert data into Datetimes table
for index, row in datetimes.iterrows():
    cursor.execute("INSERT INTO Datetimes (datetime) VALUES (%s)", (row['datetime'],))
conn.commit()

# Insert data into Conditions table
for index, row in conditions.iterrows():
    cursor.execute("INSERT INTO Conditions (conditions) VALUES (%s)", (row['conditions'],))
conn.commit()

# Insert data into Icons table
for index, row in icons.iterrows():
    cursor.execute("INSERT INTO Icons (icon) VALUES (%s)", (row['icon'],))
conn.commit()


# Insert data into Weather table
for index, row in df_weather.iterrows():
    row_values = (
        row['datetime_id'], row['condition_id'], row['icon_id'], row['temp'], row['feelslike'], row['dew'], row['humidity'], row['precip'],
        row['precipprob'], row['snow'], row['snowdepth'], row['windgust'], row['windspeed'], row['winddir'],
        row['sealevelpressure'], row['cloudcover'], row['visibility'], row['solarradiation'], row['solarenergy'], row['uvindex'],
        row['severerisk'], row['stations']
    )
    
    # Print the row values before executing the SQL statement
    print(f"Inserting row {index + 1}/{len(df_weather)}: {row_values}")
    
    # Print the SQL query for debugging
    sql_query = """
    INSERT INTO Weather (datetime_id, condition_id, icon_id, temp, feelslike, dew, humidity, precip, precipprob, snow, snowdepth, windgust, windspeed, winddir, sealevelpressure, cloudcover, visibility, solarradiation, solarenergy, uvindex, severerisk, stations)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    print(f"SQL Query: {sql_query}")
    print(f"Parameters: {row_values}")
    
    try:
        cursor.execute(sql_query, row_values)
    except mysql.connector.Error as e:
        print(f"Error inserting row {index + 1}: {e}")
        break
conn.commit()


Inserting row 1/743: (1, 1, 1, 1.7, 1.7, -11.2, 37.92, 0.0, 0, 0, 0, 18.4, 0.6, 359, 1031.8, 0.0, 16.0, 0, 0.0, 0, 10, '72505394728,72055399999,KLGA,KJRB,F1417,KNYC,72503014732')
SQL Query: 
    INSERT INTO Weather (datetime_id, condition_id, icon_id, temp, feelslike, dew, humidity, precip, precipprob, snow, snowdepth, windgust, windspeed, winddir, sealevelpressure, cloudcover, visibility, solarradiation, solarenergy, uvindex, severerisk, stations)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    
Parameters: (1, 1, 1, 1.7, 1.7, -11.2, 37.92, 0.0, 0, 0, 0, 18.4, 0.6, 359, 1031.8, 0.0, 16.0, 0, 0.0, 0, 10, '72505394728,72055399999,KLGA,KJRB,F1417,KNYC,72503014732')
Inserting row 2/743: (2, 1, 1, 1.1, -2.2, -8.0, 50.66, 0.0, 0, 0, 0, 25.9, 10.8, 349, 1032.6, 0.4, 16.0, 0, 0.0, 0, 10, '72505394728,72055399999,KLGA,KJRB,F1417,KNYC,72503014732')
SQL Query: 
    INSERT INTO Weather (datetime_id, condition_id, icon_id, temp, feelslike, de

In [117]:
# Close the cursor and connection
cursor.close()
conn.close()
print("Connection closed")


Connection closed
